In [1]:
#!pip install ipympl matplotlib numpy

In [2]:
# from google.colab import output
# output.enable_custom_widget_manager()

In [1]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
from math import *
from itertools import product
from IPython.display import clear_output
from itertools import chain

In [2]:
import torch
from torch import nn
from torch.autograd import Variable

In [3]:
with plt.ioff():
    fig, ax = plt.subplots()

In [4]:
from ipywidgets import AppLayout
from ipywidgets import Dropdown, FloatText, IntText, Button, Layout, HTML, HBox, VBox, Text

In [5]:
rng = np.random.default_rng(3123)
functions = {
    "ax^3+bx^2+cx+d":
    (np.vectorize(lambda x, a, b, c, d: a*x**3 + b*x**2 + c*x + d), {"a": [-3, 3], "b": [-3, 3], "c": [-3, 3], "d": [-3, 3]}),
    "x*sin(2*pi*x)":
    (np.vectorize(lambda x: x*sin(2*pi*x)), {}),
}

In [6]:
epsilon_rngs = {
    "normal":
    (lambda n, sigma, center=0: rng.normal(0, sigma, n), {"center": None, "sigma": None}),
    "uniform":
    (lambda n, ep0: rng.uniform(-ep0, ep0, n), {"ep0": None}),
}

In [7]:
def update_plot(f_name, f, f_params, N, e_rng, e_rng_name, e_params):
    global ax
    epsilon = e_rng(N, **e_params)
    x = np.linspace(-1, 1, 100)
    ax.clear()
    ax.plot(x, f(x, **f_params), "b", label=f_name)
    x_c = rng.uniform(-1, 1, N)
    # x_c = np.linspace(-1, 1, N)
    y_c = f(x_c, **f_params) + epsilon
    ax.scatter(x_c, y_c, c="r")
    ax.legend()
    ax.title.set_text(f"N: {N} ep_rng: {e_rng_name}")

In [8]:
def polynomial_regression(N):
    global ax
    data = ax.collections[0].get_offsets()
    x_ep = data.data[:, 0]
    y_ep = data.data[:, 1]
    x, y = ax.lines[0].get_data()

    A = np.vander(x_ep, int(N))
    # A @ x = y_ep
    w = np.linalg.lstsq(A, y_ep, rcond=None)[0][::-1]
    ft = np.vectorize(lambda x: np.sum(np.power(x, np.arange(N)) * w))
    Er = np.sum((ft(x) - y)**2) / 2
    # clear_output(wait=True)

    ax.plot(x, ft(x), label=f"f' - {N} - {Er:.3}")

    ax.axis([min(x) - 1, max(x) + 1, min(y) - 2, max(y) + 2])
    ax.legend()

In [9]:
def custom_f_regression(*fs):
    global ax
    data = ax.collections[0].get_offsets()
    x_ep = data.data[:, 0]
    y_ep = data.data[:, 1]
    x, y = ax.lines[0].get_data()
    fs = list(np.vectorize(eval("lambda x: " + f)) for f in fs)

    A = np.vstack([f(x_ep) for f in fs]).T
    w = np.linalg.lstsq(A, y_ep, rcond=None)[0]
    ft = np.vectorize(lambda x: np.sum(np.array([f(x) for f in fs]) * w))
    Er = np.sum((ft(x) - y)**2) / 2
    # clear_output(wait=True)

    ax.plot(x, ft(x), label=f"f' - {Er:.3}")

    ax.axis([min(x) - 1, max(x) + 1, min(y) - 2, max(y) + 2])
    ax.legend()

In [10]:
def polynomial_self_regression(N):

    # https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%BD%D0%B0%D0%B8%D0%BC%D0%B5%D0%BD%D1%8C%D1%88%D0%B8%D1%85_%D0%BA%D0%B2%D0%B0%D0%B4%D1%80%D0%B0%D1%82%D0%BE%D0%B2
    def lstsq_solve(A, y):
        Asq = A.T @ A
        y_r = A.T @ y
        return np.linalg.solve(Asq, y_r)

    global ax
    data = ax.collections[0].get_offsets()
    x_ep = data.data[:, 0]
    y_ep = data.data[:, 1]
    x, y = ax.lines[0].get_data()

    xv = np.vander(x_ep, int(N))
    # A = xv.T @ xv
    # y_r = xv.T @ y_ep
    w = lstsq_solve(xv, y_ep)[::-1]
    # w = np.linalg.solve(A, y_r)[::-1]
    ft = np.vectorize(lambda x: np.sum(np.power(x, np.arange(N)) * w))
    Er = np.sum((ft(x) - y)**2) / 2
    # clear_output(wait=True)

    ax.plot(x, ft(x), label=f"f' - {Er:.3}")

    ax.axis([min(x) - 1, max(x) + 1, min(y) - 2, max(y) + 2])
    ax.legend()

In [11]:
def neural_network_regression(EPOCHS, lr, layers, neural_num):
    global ax
    data = ax.collections[0].get_offsets()
    x_ep = data.data[:, 0]
    y_ep = data.data[:, 1]
    x, y = ax.lines[0].get_data()
    x_ep = torch.tensor(x_ep, dtype=torch.float32, requires_grad=True).unsqueeze(1)
    y_ep = torch.tensor(y_ep, dtype=torch.float32, requires_grad=False).unsqueeze(1)
    x = torch.tensor(x, dtype=torch.float32, requires_grad=False).unsqueeze(1)
    neural_num = int(neural_num)
    model = nn.Sequential(
        nn.Linear(1, neural_num),
        nn.ReLU(),
        *chain(*((nn.Linear(neural_num, neural_num), nn.ReLU()) for _ in range(int(layers)))),
        # nn.Linear(neural_num, neural_num),
        # nn.ReLU(),
        nn.Linear(neural_num, 1),
    )
    er_f = nn.MSELoss()
    optim = torch.optim.SGD(model.parameters(), lr=lr)

    for epoch in range(int(EPOCHS)):
        optim.zero_grad()
        out = model(x_ep)
        # print(x_ep.shape, out.shape, y_ep.shape)
        loss = er_f(out, y_ep)
        # loss = (out - target).mean()
        loss.backward()
        optim.step()

    ax.plot(x, model(x).detach().numpy(), label=f"f' - {loss:.3}")

    ax.axis([min(x) - 1, max(x) + 1, min(y) - 2, max(y) + 2])
    ax.legend()

In [12]:
approximation_methods = {
    "polynomial regression":
    (polynomial_regression, {"N": 2}),
    "custom f":
    (custom_f_regression, {}),
    "polynom_self":
    (polynomial_self_regression, {"N": 2}),
    "neural network":
    (neural_network_regression, {"EPOCHS": 300, "learning rate": 0.1, "layers num": 1, "neural number": 10}),
}

In [13]:
def create_param_inputs(functions: dict[str, tuple[callable, dict[str, list[float]]]]):

    function_blocks = dict()
    function_params_inputs = dict()

    for f_name, (_, params) in functions.items():

        function_box_items = list()
        function_inputs = list()

        for param_name, param_range in params.items():
            rng_btn = None
            if param_range is None:
                value = rng.uniform(0, 1)
            elif isinstance(param_range, list):
                value = rng.uniform(*param_range)
                rng_btn = Button(description="Generate")
            else:
                value = param_range
            inp = FloatText(
                value = value,
                step = .1,
                description=param_name,
                layout=Layout(width="95%")
            )
            function_inputs.append(inp)
            box_items = list()
            box_items.append(inp)

            if rng_btn is not None:
                rng_btn.linked_input = inp
                rng_btn.linked_input_range = param_range
                # rng_btn.on_click(lambda x: inp.__setattr__("value", rng.uniform(*param_range)))
                rng_btn.on_click(lambda x: x.linked_input.__setattr__("value", rng.uniform(*x.linked_input_range)))
                box_items.append(rng_btn)
            
            box = HBox(
                box_items
            )
            function_box_items.append(box)
        
        function_blocks[f_name] = VBox(
            function_box_items,
            layout=Layout(width="95%", display="none")
        )
        function_params_inputs[f_name] = function_inputs
    
    return function_blocks, function_params_inputs            


In [14]:
first_rng = next(iter(epsilon_rngs.keys()))
ep_rng_dropdown = Dropdown(
    options=epsilon_rngs.keys(),
    value=first_rng,
    layout=Layout(width="95%"),
    description="ε_rng",
    style={"description_width": "initial"},
)

ep_rng_blocks, ep_rng_params = create_param_inputs(epsilon_rngs)

def on_dropdown_change(value, boxes: dict[str, VBox]):
    old_box = boxes[value["old"]]
    new_box = boxes[value["new"]]
    old_box.layout.display="none"
    new_box.layout.display="block"

ep_rng_dropdown.observe(lambda value: on_dropdown_change(value, ep_rng_blocks), names="value")
ep_rng_blocks[first_rng].layout.display = "block"

first_function = next(iter(functions.keys()))
function_dropdown = Dropdown(
    options=functions.keys(),
    value=first_function,
    layout=Layout(width="100%"),
    description="function",
    style={"description_width": "initial"},
)
functions_blocks, functions_params = create_param_inputs(functions)
function_dropdown.observe(lambda value: on_dropdown_change(value, functions_blocks), names="value")
functions_blocks[first_function].layout.display="block"

first_approx = next(iter(approximation_methods.keys()))
approx_dropdown = Dropdown(
    options=approximation_methods.keys(),
    value=first_approx,
    layout=Layout(width="95%"),
    description="appr method",
    style={"description_width": "initial"},
)
approx_blocks, approx_params = create_param_inputs(approximation_methods)
approx_dropdown.observe(lambda value: on_dropdown_change(value, approx_blocks), names="value")
approx_blocks[first_approx].layout.display="block"


n_input = IntText(
                value=20,
                min=1,
                max=100,
                step=1,
                layout=Layout(width="95%"),
                description="N",
                style={"description_width": "initial"},
)


AppLayout(
    center=fig.canvas,
    header=VBox(
        [
            update_bt := Button(
                description="generate points",
                layout=Layout(width="98%"),
                style={"description_width": "initial"},
            ),
        ]
    ),
    footer=VBox(
        [
            function_dropdown,
            *functions_blocks.values()
        ]
    ),
    right_sidebar=VBox(
        [
            n_input,
            ep_rng_dropdown,
            *ep_rng_blocks.values()
        ],
        layout=Layout(height="100%", width="95%")
    ),
    left_sidebar=VBox(
        [
            create_approx_bt := Button(
                description="Create approximation function",
                layout=Layout(width="95%"),
                style={"description_width": "initial"},
            ),
            approx_dropdown,
            *approx_blocks.values()
        ],
        layout=Layout(height="100%", width="100%")
    ),
    pane_heights=["80px", 3, 1],
    pane_widths=[1,3,1]
)

AppLayout(children=(VBox(children=(Button(description='generate points', layout=Layout(width='98%'), style=But…

In [15]:
def rm_widget(bt):
    bt.rm_idx
    approx_params["custom f"].pop(bt.rm_idx)
    vbox: VBox = approx_blocks["custom f"]
    vbox.children = vbox.children[:bt.rm_idx] + vbox.children[bt.rm_idx + 1:]
    for i, c in enumerate(vbox.children[bt.rm_idx:-1]):
        c.children[0].description = f"f{bt.rm_idx + i + 1}"
        c.children[1].rm_idx -= 1


vbox: VBox = approx_blocks["custom f"]
vbox.children=[
        txt := Text(description="f1", layout=Layout(width="98%")),
        add_f_bt := Button(description="+", layout=Layout(width="98%"))
    ]
approx_params["custom f"] = [txt]

def on_add_f_click(x):
    vbox: VBox = approx_blocks["custom f"]
    l = len(vbox.children)
    layout_items = Layout(width="85%")
    hbox = HBox(
        [
            txt := Text(description=f"f{l}", layout=layout_items),
            rm_f_bt := Button(description="-", layout=layout_items),
        ],
    )
    rm_f_bt.rm_idx = l - 1
    rm_f_bt.on_click(rm_widget)
    vbox.children = vbox.children[:l - 1] + (hbox, vbox.children[-1])
    approx_params["custom f"].append(txt)
add_f_bt.on_click(on_add_f_click)

In [119]:
update_bt.on_click(lambda x: update_plot(
    function_dropdown.value,
    functions[function_dropdown.value][0],
    {w.description: w.value for w in functions_params[function_dropdown.value]},
    n_input.value,
    epsilon_rngs[ep_rng_dropdown.value][0],
    ep_rng_dropdown.value,
    {w.description: w.value for w in ep_rng_params[ep_rng_dropdown.value]},
))
# linear_regression_bt.on_click(lambda x: linear_regression())
create_approx_bt.on_click(lambda x:
    approximation_methods[approx_dropdown.value][0](
        *[w.value for w in approx_params[approx_dropdown.value]]
    )
)